# 13.7 Authenticating with Twitter Via Tweepy 
* A **Tweepy `API` object** is your gateway to using the Twitter APIs
* Must first **authenticate with Twitter**

In [ ]:
import tweepy

In [ ]:
# before executing this cell, ensure that your copy of keys.py 
# contains your Twitter credentials as described earlier
import keys  

### Creating and Configuring an `OAuthHandler` to Authenticate with Twitter

In [ ]:
auth = tweepy.OAuthHandler(keys.consumer_key,
                           keys.consumer_secret)

In [ ]:
auth.set_access_token(keys.access_token,
                      keys.access_token_secret)

### Creating the Tweepy API Object 
* `auth` is the `OAuthHandler` 
* `wait_on_rate_limit=True` tells Tweepy to **wait 15 minutes** each time it reaches a given API method’s rate limit&mdash;**prevents violations**
* `wait_on_rate_limit_notify=True` tells Tweepy to display a command-line message if you hit a rate limit

In [ ]:
api = tweepy.API(auth, wait_on_rate_limit=True, 
                 wait_on_rate_limit_notify=True)

# 13.8 Getting Information About a Twitter Account
* `API` object’s **`get_user` method** returns a **`tweepy.models.User` object** containing information about a specific user’s Twitter account 

In [ ]:
nasa = api.get_user('nasa')

* Calls the Twitter API’s [`users/show` method](https://developer.twitter.com/en/docs/accounts-and-users/follow-search-get-users/api-reference/get-users-show)
* Currently can call **up to 900 times every 15 minutes**
* **`tweepy.models` classes** correspond to returned **JSON objects**
* **`User` class** corresponds to a Twitter [**user object**](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/user-object)
* `tweepy.models` classes can **read JSON** and **turn it into Tweepy objects** 

### Getting Basic Account Information 

In [ ]:
nasa.id  # account ID created when the user joined Twitter

In [ ]:
nasa.name  # name associated with the user’s account

In [ ]:
nasa.screen_name  # user’s Twitter handle

In [ ]:
nasa.description  # description from the user’s profile

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

### Getting the Most Recent Status Update
* `User` object’s **`status` property** returns a **`tweepy.models.Status`** object
* Corresponds to a Twitter [**tweet object**](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object)

In [ ]:
nasa.status.text  # most recent tweet's text

* `…` indicates **truncated** tweet text
* **`extended_tweet` property** for tweets between 141 and 280 characters (as of Nov. 2017) 
* **Retweeting** often results in truncation

### Getting the Number of Followers 

In [ ]:
nasa.followers_count

### Getting the Number of Friends an Account Follows

In [ ]:
nasa.friends_count

### Getting Your Own Account’s Information
>```python
me = api.me()  # get User object for the authenticated account
```

# 13.9 Introduction to Tweepy Cursors: Getting an Account’s Followers and Friends 
* Twitter API methods often return collections of objects
    * Tweets in **your Twitter timeline**
    * Tweets in **another account’s timeline**
    * Lists of tweets that match specified search criteria 
* **Timeline** &mdash; tweets sent by a user and by that user’s friends
* Each method’s docs specify max items returned by one call—a **page** of results
* JSON responses say **whether there are more pages to get**

# 13.9 Introduction to Tweepy Cursors: Getting an Account’s Followers and Friends (cont.)
* A **`Cursor`** handles **paging** 
* Invokes a method and **checks if there's another page of results**
* If so, automatically calls the method again  
* Continues, subject to rate limits, until there are no more results to process
* If `API` object configured to **wait on rate limits**, `Cursor`s wait as needed between calls
* [Tweepy `Cursor` tutorial](http://docs.tweepy.org/en/latest/cursor_tutorial.html)

## 13.9.1 Determining an Account’s Followers  
* Via the `API` object’s **`followers` method**
* Calls Twitter’s [`followers/list` method](https://developer.twitter.com/en/docs/accounts-and-users/follow-search-get-users/api-reference/get-followers-list.html)
* Returns groups of 20 by default
* Can request up to 200 at a time 
* For demonstration purposes, we’ll grab 10 of NASA’s followers. 

### Creating a Cursor 
* Will call the `followers` method for NASA’s account

In [ ]:
followers = []  # for storing followers' User objects

In [ ]:
cursor = tweepy.Cursor(api.followers, screen_name='nasa')

* First argument is **name of Tweepy method to call**
* Additional keyword arguments are passed to method named in first argument

### Get Results 
* Iterate over the `Cursor`'s results  
* Cursor’s **`items` method** calls `api.followers` and returns the `follower`s method’s results

In [ ]:
for account in cursor.items(10):  # request only 10 results
    followers.append(account.screen_name)

* Display followers in ascending alphabetical order

In [ ]:
print('Followers:', 
      ' '.join(sorted(followers, key=lambda s: s.lower())))

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

### Automatic Paging
* To get up to 200 followers at a time, create the `Cursor` with the `count` keyword argument
>```python
cursor = tweepy.Cursor(api.followers, screen_name='nasa', count=200)
```
* **`items`** with no argument attempts to get **all** followers
    * Could **take significant time** due to rate limits
    * Twitter’s **`followers/list`** returns max of 200 followers and allows only 15 calls every 15 minutes
        * 3000 followers every 15 minutes 
        * NASA has 30+ million followers

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

### Getting Follower IDs Rather Than Followers
* Can get **many more Twitter IDs** by calling **`followers_ids` method**
* Calls Twitter’s [`followers/ids` method](https://developer.twitter.com/en/docs/accounts-and-users/follow-search-get-users/api-reference/get-followers-ids)
* Returns up to **5000 ID numbers at a time**
* Invoke up to 15 times every 15 minutes for **75,000 account IDs per rate-limit interval**
* Particularly useful when combined with the API object’s **`lookup_users` method**, which calls Twitter’s [`users/lookup` method](https://developer.twitter.com/en/docs/accounts-and-users/follow-search-get-users/api-reference/get-friends-list) 
    * Can return up to **100 `User` objects** at a time and can be called up to 300 times every 15 minutes 
    * Using this combination, could get up to **30,000 `User` objects per rate-limit interval**

<hr style="height:2px; border:none; color:#AAA; background-color:#AAA;">

## 13.9.2 Determining Whom an Account Follows 
* Via the `API` object's **`friends` method**
* Calls the Twitter’s [`friends/list` method](https://developer.twitter.com/en/docs/accounts-and-users/follow-search-get-users/api-reference/get-friends-list) 
* Returns a list of **`User` objects**
* Groups of 20 by default, but can get up to 200 at a time
* Can call up to 15 times every 15 minutes

In [ ]:
friends = []

In [ ]:
cursor = tweepy.Cursor(api.friends, screen_name='nasa')

In [ ]:
for friend in cursor.items(10):
    friends.append(friend.screen_name)

In [ ]:
print('Friends:', 
      ' '.join(sorted(friends, key=lambda s: s.lower())))

## 13.9.3 Getting a User’s Recent Tweets 
* Via `API` object's **`user_timeline`** method 
* **Timeline** includes account’s tweets and tweets from that account’s friends 
* Calls Twitter’s [`statuses/user_timeline` method](https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-user_timeline)
* Returns most recent 20 tweets as `Status` objects, but can return up to 200 at a time 
* Can access an account’s **3200 most recent tweets** 
* May call it up to 1500 times every 15 minutes

## 13.9.3 Getting a User’s Recent Tweets (cont.)

In [ ]:
# should use Cursor if getting more than max tweets per call
nasa_tweets = api.user_timeline(screen_name='nasa', count=3)

In [ ]:
for tweet in nasa_tweets:
    print(f'{tweet.user.screen_name}: {tweet.text}\n')

### Get Recent Tweets from Your Own Timeline
* Via `API` method **`home_timeline`**
>```python
api.home_timeline()
```
* Calls Twitter’s [`statuses/home_timeline` method](https://developer.twitter.com/en/docs/tweets/timelines/api-reference/get-statuses-home_timeline). 
* Most recent 20 tweets, but can get up to 200 at a time

# 13.10 Searching Recent Tweets 
* Via `API` method **`search`** 
* Returns tweets that **match a query string**
* Only for the **previous seven days’ tweets**
* Not guaranteed to return all matching tweets
* Calls Twitter’s `search/tweets` method
* Returns 15 tweets at a time by default, but can return up to 100

### Utility Function `print_tweets` from `tweetutilities.py`
* Receives the results of a call to API method `search` and for each tweet displays the user’s `screen_name` and the tweet’s `text`. 
* If the tweet is not in English and the `tweet.lang` is not `'und'` (undefined), we’ll also translate the tweet to English  

In [ ]:
# Instructor Note: We modified this function in tweetutilities.py 
# to catch exceptions when a tweet cannot be translated
from tweetutilities import print_tweets

```python
def print_tweets(tweets):
    """For each Tweepy Status object in tweets, display the 
    user's screen_name and tweet text. If the language is not
    English, translate the text with TextBlob."""
    for tweet in tweets:
        print(f'{tweet.screen_name}:', end=' ')
    
        if 'en' in tweet.lang:
            print(f'{tweet.text}\n')
        elif 'und' not in tweet.lang:  # translate to English first
            print(f'\n  ORIGINAL: {tweet.text}')
            print(f'TRANSLATED: {TextBlob(tweet.text).translate()}\n')

```

### Searching for Specific Words
* **`q` keyword argument** specifies the **query string**
* Should use **`Cursor`** for more than max results

In [ ]:
tweets = api.search(q='Mars Opportunity Rover', count=3)

In [ ]:
print_tweets(tweets) 

### Searching with Twitter Search Operators
* Use Twitter search operators to refine search results
* The following table shows several Twitter search operators. 
* Multiple operators can be combined to construct more complex queries. 
* [For all the operators, click the `operators` link here](https://twitter.com/search-home)

| Example&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;	| Finds tweets containing
| :---	| :---
| `python twitter` 	| Implicit _logical and_ operator—Finds tweets containing `python` _and_ `twitter`.
| `python OR twitter` 	| Logical `OR` operator—Finds tweets containing `python` or `twitter` or both.
| `python ?` 	| `?` (question mark)—Finds tweets asking questions about `python`.
| `planets -mars` 	| `-` (minus sign)—Finds tweets containing `planets` but not `mars`.
| `python :)` 	| `:)` (happy face)—Finds _positive sentiment_ tweets containing `python`.
| `python :(` 	| `:(` (sad face)—Finds _negative sentiment_ tweets containing `python`.
| `since:2018-09-01` 	| Finds tweets _on or after_ the specified date, which must be in the form `YYYY-MM-DD`.
| `near:"New York City" `	| Finds tweets that were sent near `"New York City"`.
| `from:nasa` 	| Finds tweets from the account `@nasa`.
| `to:nasa` 	| Finds tweets to the account `@nasa`.

### Searching with Twitter Search Operators (cont.)
* **Use a date within seven days before you execute this code**

In [ ]:
tweets = api.search(q='from:nasa since:2019-06-06', count=3)

In [ ]:
print_tweets(tweets)

### Searching for a Hashtag
* A **hashtag marked with `#`** indicates something of importance, like a trending topic

In [ ]:
tweets = api.search(q='#collegefootball', count=2)

In [ ]:
print_tweets(tweets)

# 13.11 Spotting Trends: Twitter Trends API
* **“Going viral”** &mdash; thousands or millions of people tweeting at once 
* Twitter maintains a list of **trending topics** worldwide 
* **Twitter Trends API** can return lists of trending-topic locations and the top 50 trending topics for each location

## 13.11.1 Places with Trending Topics 
* Tweepy `API`’s **`trends_available` method** calls Twitter’s [`trends/available`](https://developer.twitter.com/en/docs/trends/locations-with-trending-topics/api-reference/get-trends-available)  
* Returns **list of dictionaries** representing locations 

In [ ]:
trends_available = api.trends_available()

In [ ]:
len(trends_available)

* Each element contains location’s `name`, `woeid` (**Yahoo! Where on Earth ID**) and more

In [ ]:
trends_available[0]

In [ ]:
trends_available[1]

## 13.11.1 Places with Trending Topics (cont.)
* WOEID 1 represents **worldwide** 
* WOEID values for several landmarks, cities, states and continents

| Place | WOEID | Place | WOEID
| :---	| :--- | :---	| :---
| Statue of Liberty | 23617050 | Iguazu Falls| 468785
| Washington, D.C.| 2514815 | United States| 23424977
| Paris, France| 615702 | Europe| 24865675

* Also can search for locations close to a **latitude** and **longitude** via the **Tweepy `API`’s `trends_closest` method**
* Calls Twitter's [`trends/closest` method](https://developer.twitter.com/en/docs/trends/locations-with-trending-topics/api-reference/get-trends-closest)

## 13.11.2 Getting a List of Trending Topics 
* Via Tweepy `API`’s **`trends_place` method** 
* Calls **Twitter Trends API’s [`trends/place` method](https://developer.twitter.com/en/docs/trends/trends-for-location/api-reference/get-trends-place)**
* Returns top 50 trending topics for the location 
* [Look up WOEIDs](http://www.woeidlookup.com) 
* Look up WOEID’s programmatically using **Yahoo!’s web services** via [Python libraries like `woeid`](https://github.com/Ray-SunR/woeid)

### Worldwide Trending Topics 

In [ ]:
world_trends = api.trends_place(id=1)  # list containing one dictionary

* **`'trends'` key** refers to a list of dictionaries representing each trend

In [ ]:
trends_list = world_trends[0]['trends']

* Each trend has **`name`**, **`url`**, **`promoted_content`** (whether it's an advertisement), **`query`** and **`tweet_volume`** keys

In [ ]:
trends_list[0]

### Get Today's Worldwide Trending Topics (cont.)
* For **trends with more than 10,000 tweets**, the `tweet_volume` is the number of tweets; otherwise, it’s `None`
* Filter the list so that it contains only trends with more than 10,000 tweets:

In [ ]:
trends_list = [t for t in trends_list if t['tweet_volume']]

* Sort the trends in _descending_ order by `tweet_volume`:

In [ ]:
from operator import itemgetter 

In [ ]:
trends_list.sort(key=itemgetter('tweet_volume'), reverse=True) 

### Get Today's Worldwide Trending Topics (cont.)
* Display names of the **top five trending topics**

In [ ]:
for trend in trends_list[:5]:
    print(trend['name'])

### New York City Trending Topics (WOEID `2459115`)

In [ ]:
nyc_trends = api.trends_place(id=2459115)  # New York City WOEID

In [ ]:
nyc_list = nyc_trends[0]['trends']

In [ ]:
nyc_list = [t for t in nyc_list if t['tweet_volume']]

In [ ]:
nyc_list.sort(key=itemgetter('tweet_volume'), reverse=True) 

In [ ]:
for trend in nyc_list[:5]:
    print(trend['name'])

## 13.11.3 Create a Word Cloud from Trending Topics
* Visualize New York City’s trending topics with more than 10,000 tweets each

In [ ]:
topics = {}  # dictionary to store trend names and volumes 

In [ ]:
for trend in nyc_list:
    topics[trend['name']] = trend['tweet_volume']

## 13.11.3 Create a Word Cloud from Trending Topics (cont.)
* `prefer_horizontal=0.5` **suggests** that 50% of the words should be horizontal, but may ignore to fit content

In [ ]:
from wordcloud import WordCloud

In [ ]:
wordcloud = WordCloud(width=1600, height=900,
    prefer_horizontal=0.5, min_font_size=10, colormap='prism', 
    background_color='white')

In [ ]:
wordcloud = wordcloud.fit_words(topics)

In [ ]:
wordcloud = wordcloud.to_file('TrendingTwitter.png')

* **Instructor note: You might need to duplicate the cell below (select it in the margin then press `c` to copy it and `v` to paste it), then execute the cell to see the new word cloud**
* Depending on the number of trending topics, the word cloud may be sparse

![A word cloud generated from trending Twitter hashtags](./TrendingTwitter.png "A word cloud generated from trending Twitter hashtags")

------
&copy;1992&ndash;2020 by Pearson Education, Inc. All Rights Reserved. This content is based on Chapter 5 of the book [**Intro to Python for Computer Science and Data Science: Learning to Program with AI, Big Data and the Cloud**](https://amzn.to/2VvdnxE).

DISCLAIMER: The authors and publisher of this book have used their 
best efforts in preparing the book. These efforts include the 
development, research, and testing of the theories and programs 
to determine their effectiveness. The authors and publisher make 
no warranty of any kind, expressed or implied, with regard to these 
programs or to the documentation contained in these books. The authors 
and publisher shall not be liable in any event for incidental or 
consequential damages in connection with, or arising out of, the 
furnishing, performance, or use of these programs.                  